In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('himachal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B9B86CB5+28821]
	(No symbol) [0x00007FF7B9AF3840]
	(No symbol) [0x00007FF7B999578A]
	(No symbol) [0x00007FF7B99E91BE]
	(No symbol) [0x00007FF7B99E94AC]
	(No symbol) [0x00007FF7B9A32647]
	(No symbol) [0x00007FF7B9A0F33F]
	(No symbol) [0x00007FF7B9A2F412]
	(No symbol) [0x00007FF7B9A0F0A3]
	(No symbol) [0x00007FF7B99DA778]
	(No symbol) [0x00007FF7B99DB8E1]
	GetHandleVerifier [0x00007FF7B9EBFCAD+3408013]
	GetHandleVerifier [0x00007FF7B9ED741F+3504127]
	GetHandleVerifier [0x00007FF7B9ECB5FD+3455453]
	GetHandleVerifier [0x00007FF7B9C4BDBB+835995]
	(No symbol) [0x00007FF7B9AFEB5F]
	(No symbol) [0x00007FF7B9AFA814]
	(No symbol) [0x00007FF7B9AFA9AD]
	(No symbol) [0x00007FF7B9AEA199]
	BaseThreadInitThunk [0x00007FFFBA46257D+29]
	RtlUserThreadStart [0x00007FFFBB40AF08+40]

Error occurred while scraping bus details for https://

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 591,A/C Executive (2+3),21:00,10h 20m,07:20,3.2,INR 632,42 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 570,Ordinary 3+2 Non AC Seater,21:25,09h 20m,06:45,3.2,INR 512,34 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 9,Himsuta AC Seater Volvo/Scania 2+2,22:00,09h 30m,07:30,3.8,INR 912,27 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 579,Ordinary 3+2 Non AC Seater,22:30,09h 45m,08:15,4.3,INR 512,26 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 22,Himsuta AC Seater Volvo/Scania 2+2,23:00,08h 50m,07:50,4.3,INR 912,7 Seats available
...,...,...,...,...,...,...,...,...,...,...
331,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...,HRTC - 1107,Ordinary 3+2 Non AC Seater,20:30,07h 45m,04:15,3.6,INR 488,35 Seats available
332,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...,HRTC - 78,Himsuta AC Seater Volvo/Scania 2+2,21:15,07h 45m,05:00,3.9,INR 982,30 Seats available
333,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...,Bedi Travels,Volvo A/C Semi Sleeper (2+2),23:15,07h 15m,06:30,4.9,675,10 Seats available
334,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),23:30,07h 10m,06:40,4.4,665,7 Seats available
